In [80]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import date
import creds
from kaggle.api.kaggle_api_extended import KaggleApi
import datetime

In [16]:
wiki_url = 'https://en.wikipedia.org/wiki/The_Simpsons_(season_33)'

In [17]:
imdb_url = 'https://www.imdb.com/title/tt0096697/episodes?season=33'

# Wikipedia

In [18]:
req=requests.get(wiki_url)
content=req.text

In [19]:
soup=BeautifulSoup(content)

In [20]:
rows=soup.findAll('tr', 'vevent')

In [21]:
rows[1]

<tr class="vevent" style="text-align:center;background:#F2F2F2"><th id="ep708" rowspan="1" scope="row" style="text-align:center">708</th><td style="text-align:center">2</td><td class="summary" rowspan="1" style="text-align:left">"<a href="/wiki/Bart%27s_in_Jail!" title="Bart's in Jail!">Bart's in Jail!</a>"</td><td style="text-align:center"><a href="/wiki/Steven_Dean_Moore" title="Steven Dean Moore">Steven Dean Moore</a></td><td style="text-align:center">Nick Dahan</td><td style="text-align:center">October 3, 2021<span style="display:none"> (<span class="bday dtstart published updated">2021-10-03</span>)</span></td><td id="pcQABF18" rowspan="1" style="text-align:center">QABF18</td><td style="text-align:center">1.48<sup class="reference" id="cite_ref-33.02_14-0"><a href="#cite_note-33.02-14">[14]</a></sup></td></tr>

In [22]:
rows[0].findAll('th')[0].text

'707'

In [23]:
data = rows[9].findAll('td')

In [24]:
data

[<td style="text-align:center">11</td>,
 <td class="summary" rowspan="1" style="text-align:left">"<a href="/wiki/The_Longest_Marge" title="The Longest Marge">The Longest Marge</a>"</td>,
 <td style="text-align:center">Matthew Nastuk</td>,
 <td style="text-align:center"><a href="/wiki/Brian_Kelley_(writer)" title="Brian Kelley (writer)">Brian Kelley</a></td>,
 <td style="text-align:center">January 2, 2022<span style="display:none"> (<span class="bday dtstart published updated">2022-01-02</span>)</span></td>,
 <td id="pcUABF05" rowspan="1" style="text-align:center">UABF05</td>,
 <td style="text-align:center">2.02<sup class="reference" id="cite_ref-33.11_23-0"><a href="#cite_note-33.11-23">[23]</a></sup></td>]

In [25]:
data[6].text.split('[')[0]

'2.02'

In [26]:
"2.02".split('\[')

['2.02']

In [27]:
data[4].findAll('span')[1].text

'2022-01-02'

# IMDB

In [28]:
imdb_req=requests.get(imdb_url)
imdb_content=imdb_req.text

In [29]:
imdb_soup=BeautifulSoup(imdb_content)

In [30]:
descriptions = imdb_soup.findAll('div', 'item_description')

In [31]:
descriptions[0].text.split('\n')[1]

'Marge stages a revival of a musical from high school, but her pleasant memories are threatened by the return of her old rival.    '

In [32]:
rows=imdb_soup.findAll('div', 'ipl-rating-star small')

In [33]:
len(rows)

22

In [34]:
rows[1].findAll('span', 'ipl-rating-star__rating')[0].text

'7.0'

In [35]:
"\"hello\""

'"hello"'

In [36]:
"\"hello\"".strip("\"")

'hello'

# The Movie DB

In [37]:
SERIES_ID = 456
import json

In [47]:
url = f"https://api.themoviedb.org/3/tv/{SERIES_ID}/season/{2}/episode/{1}?language=en-US&api_key={creds.TMDB_KEY}"

response = requests.get(url)

json.loads(response.text)['vote_average']
json.loads(response.text)['vote_count']

46

# Data Construction

In [82]:
#initialize dataframes
episode_data = []

In [83]:
# For each season
seasons = list(range(1,35))
for season in seasons:
    print(season)
    wiki = f'https://en.wikipedia.org/wiki/The_Simpsons_(season_{season})'
    imdb = f'https://www.imdb.com/title/tt0096697/episodes?season={season}'
    
    
    #wiki
    req=requests.get(wiki)
    content=req.text
    soup=BeautifulSoup(content)
    rows=soup.findAll('tr', 'vevent')
    
    #imdb
    imdb_req=requests.get(imdb)
    imdb_content=imdb_req.text
    imdb_soup=BeautifulSoup(imdb_content)
    imdb_rows=imdb_soup.findAll('div', 'ipl-rating-star small')
    descriptions = imdb_soup.findAll('div', 'item_description')
    

    
    for index in range(len(rows)):
        ep_row = {}
        ep_row['season'] = season
        data = rows[index].findAll('td')
        ep_row['number_in_series'] = rows[index].findAll('th')[0].text.split('[')[0]
        ep_row['number_in_season'] = data[0].text
        ep_row['title'] = data[1].text.strip("\"")
        ep_row['directed_by'] = data[2].text.split('[')[0]
        ep_row['written_by'] = data[3].text.split('[')[0]
        if len(data[4].findAll('span')) > 1:
            ep_row['original_air_date'] = data[4].findAll('span')[1].text
        ep_row['production_code'] = data[5].text.split("[")[0]
        if len(data[6].text.split('[')) > 0:
            ep_row['us_viewers_in_millions'] = data[6].text.split('[')[0]
        if index < len(descriptions):
            ep_row['description'] = descriptions[index].text.split('\n')[1]

        tmdb_url = f"https://api.themoviedb.org/3/tv/{SERIES_ID}/season/{season}/episode/{data[0].text}?language=en-US&api_key={creds.TMDB_KEY}"

        response = requests.get(tmdb_url)
        output = json.loads(response.text)
        ep_row['tmdb_rating'] = str(output['vote_average'])
        ep_row['tmdb_vote_count'] = str(output['vote_count'])

        #need imdb rating
        if index < len(imdb_rows):
            if len(imdb_rows[index].findAll('span', 'ipl-rating-star__rating')) > 0:
                rating = imdb_rows[index].findAll('span', 'ipl-rating-star__rating')[0].text
                ep_row['imdb_rating'] = rating
        
        
        episode_data.append(ep_row)

        
        
episode_data = pd.DataFrame(episode_data, columns = ['title', 'description', 'original_air_date', 
                                                     'production_code','directed_by', 'written_by', 'season', 'number_in_season', 
                                                     'number_in_series', 'us_viewers_in_millions', 'imdb_rating', 'tmdb_rating', 'tmdb_vote_count'])         

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


In [87]:
#write data
episode_data.index.name = 'id'

In [88]:
episode_data.season.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34])

In [89]:
episode_data.tail()

,title,description,original_air_date,production_code,directed_by,written_by,season,number_in_season,number_in_series,us_viewers_in_millions,imdb_rating,tmdb_rating,tmdb_vote_count
id,,,,,,,,,,,,,
742,Fan-ily Feud,Homer publicly disparages a pop singer and fac...,2023-04-23,OABF11,Timothy Bailey,Broti Gupta,34,18,746,1.00,5.2,4.5,2
743,Write Off This Episode,Marge is seduced by the money and prestige of ...,2023-04-30,OABF12,Matthew Nastuk,J. Stewart Burns,34,19,747,0.89,5.8,6.5,2
744,The Very Hungry Caterpillars,The Simpsons are forced to spend time with eac...,2023-05-07,OABF14,Gabriel DeFrancesco,Brian Kelley,34,20,748,0.82,6.7,6.0,2
745,Clown V. Board of Education,"Krusty opens a school for clowning, which beco...",2023-05-14,OABF15,Lance Kramer,Jeff Westbrook,34,21,749,0.77,6.3,6.0,2
746,Homer's Adventures Through the Windshield Glass,Time stands still for Homer when he flies thro...,2023-05-21,OABF13,Bob Anderson,Tim Long,34,22,750,0.87,5.9,0.0,1


In [107]:
prev_episode_data = pd.read_csv('./data/simpsons_episodes.csv', index_col='id')

In [113]:
SAME = prev_episode_data.shape == episode_data.shape

In [114]:
SAME

True

In [93]:
if not SAME:
    episode_data.to_csv('./data/simpsons_episodes.csv')
    api = KaggleApi()
    api.authenticate()

    api.dataset_create_version(
    "./data/",
    version_notes=f"Updated on {datetime.datetime.now().strftime('%Y-%m-%d')}",
    )

Starting upload for file simpsons_characters.csv


100%|██████████| 210k/210k [00:01<00:00, 198kB/s] 


Upload successful: simpsons_characters.csv (210KB)
Starting upload for file simpsons_episodes.csv


100%|██████████| 178k/178k [00:00<00:00, 202kB/s] 


Upload successful: simpsons_episodes.csv (178KB)


https://www.kaggle.com/jonbown/simpsons-episodes-2016